In [78]:
import pandas as pd   
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Features Engineering and Selection

In [79]:
df = pd.read_csv("D:\data prep\dseb63_final_project_DP_dataset\dseb63_final_project_DP_dataset\dseb63_bureau.csv")
df.head()

,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,SK_ID_CURR
0,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,254629
1,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,254629
2,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,254629
3,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN,254629
4,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,254629


In [80]:
#Drop cột có distribution chênh lệch và cột có correlation cao
new_df = df.drop(columns=['CREDIT_CURRENCY','DAYS_ENDDATE_FACT', 'DAYS_CREDIT_UPDATE'])

In [81]:
#Drop duplicated SK_ID_CURR rows
new_df = new_df.drop_duplicates(subset='SK_ID_CURR')

Subtract DAYS_CREDIT_ENDDATE from DAYS_CREDIT to get the duration of the credit.

In [82]:
new_df['CREDIT_DURATION'] = new_df['DAYS_CREDIT_ENDDATE'] - new_df['DAYS_CREDIT']

Create a new feature that represents the ratio of AMT_CREDIT_SUM_DEBT to AMT_CREDIT_SUM. This could give an indication of how much of the credit has been utilized.

In [83]:
new_df['CREDIT_USAGE'] = new_df['AMT_CREDIT_SUM_DEBT'] / new_df['AMT_CREDIT_SUM']

Calculate the percentage of overdue days by dividing CREDIT_DAY_OVERDUE by DAYS_CREDIT. This could provide insights into the severity of credit defaults.

In [84]:
new_df['OVERDUE_PERCENTAGE'] = new_df['CREDIT_DAY_OVERDUE'] / new_df['DAYS_CREDIT']

Create a binary indicator (0 or 1) for whether the credit is active (CREDIT_ACTIVE equals "Active").

In [85]:
new_df['ACTIVE_CREDIT_INDICATOR'] = (new_df['CREDIT_ACTIVE'] == 'Active').astype(int)

Count the number of missing values in each row. This might capture patterns related to incomplete information.

In [86]:
new_df['MISSING_VALUES'] = new_df.isnull().sum(axis=1)

Create a feature that represents the total number of previous loans for each SK_ID_CURR

In [87]:
new_df['NUM_PREVIOUS_LOANS'] = new_df.groupby('SK_ID_CURR')['SK_ID_BUREAU'].transform('count')

Create a new feature that represents the ratio of AMT_CREDIT_SUM_DEBT to AMT_CREDIT_SUM.

In [88]:
new_df['DEBT_CREDIT_RATIO'] = new_df['AMT_CREDIT_SUM_DEBT'] / new_df['AMT_CREDIT_SUM']

Create a binary indicator (0 or 1) for whether the credit has any overdue amount (AMT_CREDIT_SUM_OVERDUE is greater than 0).

In [89]:
new_df['CREDIT_OVERDUE_INDICATOR'] = (new_df['AMT_CREDIT_SUM_OVERDUE'] > 0).astype(int)

Calculate the average AMT_CREDIT_SUM for each SK_ID_CURR.

In [90]:
new_df['AVG_CREDIT_AMOUNT_PER_CLIENT'] = new_df.groupby('SK_ID_CURR')['AMT_CREDIT_SUM'].transform('mean')

Replace -inf = NaN

In [91]:
new_df['DEBT_CREDIT_RATIO'].replace(-np.inf, np.nan, inplace=True)
new_df['CREDIT_USAGE'].replace(-np.inf, np.nan, inplace=True)
new_df['OVERDUE_PERCENTAGE'].replace(-np.inf, np.nan, inplace=True)

In [105]:
# Calculate the percentage of null values for each column
null_percentage = (new_df.isnull().sum() / len(new_df)) * 100

# Create a DataFrame to display the null percentages
null_percentage_df = pd.DataFrame({'Column': null_percentage.index, 'Null Percentage': null_percentage.values})

# Display the DataFrame sorted by null percentage in descending order
null_percentage_df = null_percentage_df.sort_values(by='Null Percentage', ascending=False)
print(null_percentage_df)

                          Column  Null Percentage
12                   AMT_ANNUITY        78.034544
5         AMT_CREDIT_MAX_OVERDUE        75.883806
9           AMT_CREDIT_SUM_LIMIT        25.684369
15                  CREDIT_USAGE        12.556027
20             DEBT_CREDIT_RATIO        12.556027
8            AMT_CREDIT_SUM_DEBT         8.167641
4            DAYS_CREDIT_ENDDATE         2.731403
14               CREDIT_DURATION         2.731403
22  AVG_CREDIT_AMOUNT_PER_CLIENT         0.000759
7                 AMT_CREDIT_SUM         0.000759
16            OVERDUE_PERCENTAGE         0.000759
21      CREDIT_OVERDUE_INDICATOR         0.000000
19            NUM_PREVIOUS_LOANS         0.000000
18                MISSING_VALUES         0.000000
17       ACTIVE_CREDIT_INDICATOR         0.000000
0                   SK_ID_BUREAU         0.000000
13                    SK_ID_CURR         0.000000
1                  CREDIT_ACTIVE         0.000000
10        AMT_CREDIT_SUM_OVERDUE         0.000000


In [92]:
new_df.head()

,SK_ID_BUREAU,CREDIT_ACTIVE,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,...,SK_ID_CURR,CREDIT_DURATION,CREDIT_USAGE,OVERDUE_PERCENTAGE,ACTIVE_CREDIT_INDICATOR,MISSING_VALUES,NUM_PREVIOUS_LOANS,DEBT_CREDIT_RATIO,CREDIT_OVERDUE_INDICATOR,AVG_CREDIT_AMOUNT_PER_CLIENT
0,5714462,Closed,-497,0,-153.0,NaN,0,91323.00,0.000,NaN,...,254629,344.0,0.000000,-0.0,0,3,1,0.000000,0,91323.00
11,5714469,Closed,-1896,0,-1684.0,14985.0,0,76878.45,0.000,0.000,...,270608,212.0,0.000000,-0.0,0,1,1,0.000000,0,76878.45
17,5714475,Active,-96,0,269.0,0.0,0,89910.00,76905.000,0.000,...,234055,365.0,0.855355,-0.0,1,1,1,0.855355,0,89910.00
18,5714482,Closed,-318,0,-187.0,NaN,0,0.00,0.000,0.000,...,60900,131.0,NaN,-0.0,0,3,1,NaN,0,0.00
26,5714491,Active,-784,0,1008.0,0.0,0,0.00,-411.615,411.615,...,235326,1792.0,NaN,-0.0,1,1,1,NaN,0,0.00


In [93]:
#Merge With Bureau Balance
df2 = pd.read_csv("D:\data prep\dseb63_final_project_DP_dataset\dseb63_final_project_DP_dataset\dseb63_bureau_balance.csv")
df2.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [94]:
#Define a Function to Extract Mean Values for Bureau Balance Data
def extract_mean(x):
    y = x.groupby('SK_ID_BUREAU', as_index=False).mean().add_prefix('BUR_BAL_MEAN_') 
    return y

In [95]:
bureau_bal_mean = extract_mean(df2)

C:\Users\Admin\AppData\Local\Temp\ipykernel_624\3353841738.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  y = x.groupby('SK_ID_BUREAU', as_index=False).mean().add_prefix('BUR_BAL_MEAN_')


In [96]:
bureau_bal_mean = bureau_bal_mean.rename(columns = {'BUR_BAL_MEAN_SK_ID_BUREAU' : 'SK_ID_BUREAU'})

In [97]:
final = new_df.merge(bureau_bal_mean,  on = 'SK_ID_BUREAU', how = 'left')
final.drop('SK_ID_BUREAU', axis = 1, inplace = True)

In [98]:
#Apply the Second Mean Extraction Function to the Merged Dataset
def extract_mean_bur(x):
    y = x.groupby('SK_ID_CURR', as_index=False).mean().add_prefix('BUR_MEAN_')
    return y

In [99]:
final_mean_values = extract_mean_bur(final)
final_mean_values = final.rename(columns = {'BUR_MEAN_SK_ID_CURR' : 'SK_ID_CURR'})

C:\Users\Admin\AppData\Local\Temp\ipykernel_624\3243638089.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  y = x.groupby('SK_ID_CURR', as_index=False).mean().add_prefix('BUR_MEAN_')


In [100]:
sk_id_curr_column = final_mean_values['SK_ID_CURR']
final_mean_values.drop('SK_ID_CURR', axis=1, inplace=True)
final_mean_values.insert(0, 'SK_ID_CURR', sk_id_curr_column)

In [101]:
final_mean_values.head()

,SK_ID_CURR,CREDIT_ACTIVE,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,...,CREDIT_DURATION,CREDIT_USAGE,OVERDUE_PERCENTAGE,ACTIVE_CREDIT_INDICATOR,MISSING_VALUES,NUM_PREVIOUS_LOANS,DEBT_CREDIT_RATIO,CREDIT_OVERDUE_INDICATOR,AVG_CREDIT_AMOUNT_PER_CLIENT,BUR_BAL_MEAN_MONTHS_BALANCE
0,254629,Closed,-497,0,-153.0,NaN,0,91323.00,0.000,NaN,...,344.0,0.000000,-0.0,0,3,1,0.000000,0,91323.00,NaN
1,270608,Closed,-1896,0,-1684.0,14985.0,0,76878.45,0.000,0.000,...,212.0,0.000000,-0.0,0,1,1,0.000000,0,76878.45,NaN
2,234055,Active,-96,0,269.0,0.0,0,89910.00,76905.000,0.000,...,365.0,0.855355,-0.0,1,1,1,0.855355,0,89910.00,NaN
3,60900,Closed,-318,0,-187.0,NaN,0,0.00,0.000,0.000,...,131.0,NaN,-0.0,0,3,1,NaN,0,0.00,NaN
4,235326,Active,-784,0,1008.0,0.0,0,0.00,-411.615,411.615,...,1792.0,NaN,-0.0,1,1,1,NaN,0,0.00,NaN


In [102]:
final_mean_values.to_csv('bureau_balance_merge_bureau.csv', index = False)